# <a id='toc1_'></a>[JO 2024 project](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [JO 2024 project](#toc1_)    
  - [Prelude](#toc1_1_)    
  - [Imports](#toc1_2_)    
  - [Fonctions](#toc1_3_)    
  - [Data collect](#toc1_4_)    
    - [Extract one country data](#toc1_4_1_)    
    - [Extract All countries data](#toc1_4_2_)    
    - [Extract all data](#toc1_4_3_)    
    - [Description of data](#toc1_4_4_)    
  - [Transform](#toc1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Prelude](#toc0_)

Work in progress . . .  

Summarize about the project :
- Data source : <a href="https://www.olympic.org/news">International Olympic Committee</a>
- Data extract from : <a href="http://olympanalyt.com/OlympAnalytics.php">olympanalyt.com - mail : sportsencyclo@gmail.com</a>

## <a id='toc1_2_'></a>[Imports](#toc0_)

In [151]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

## <a id='toc1_3_'></a>[Fonctions](#toc0_)

In [152]:
def GetHTML(url) :
    """
    Needs : url website
    Return : HTML text
    """
    response = requests.get(url)

    if response.status_code == 200 :
        print("Response OK, continue.")
    else :
        print("Access impossible.")
    
    return response.text

In [153]:
def Getdataframe(html) :

    """
    Needs : HTML text 
            with response = requests.get(url).text
    Return : Dataframe with athlete AND sport description : medal, country, athlete_name, games, sport, event
    Requirement : Must to apply on website like : adress[COUNTRY]adress
                i.e : http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=[COUNTRY]&param_games=ALL&param_sport=ALL
    """

    # Converting text (html) in soup (text parsed)
    soup = BeautifulSoup(html, "html5lib") #parse

    # Finding the correct table according to description
    table = soup.find("table", class_="simpletable", style="text-align:left;")

    # Retrieving the list (find_all) of all rows
    rows = table.find_all("tr")

    # Creating the table (future dataframe)
    athletes = []

    # For each row (= athlete)
    for row in rows:
        #describe_athlete = row.text.split() # Get the text
        cells = row.find_all("td") # Retrieving the list (find_all) of all cells in the row

        if len(cells) == 7: # If there are 7 cells then it's an athlete
            medal = cells[0].find('img')['title'][0]  # Medal title
            country = cells[1].find('img')['title']  # Athlete's country
            athlete_name = str(cells[2]).replace("<br/>","/").replace("</td>","").replace("<td>","") # Athlete's name, splitted by "/" if it's a team
            games = cells[3].find('img')['title']  # Olympic Games
            sport = cells[4].find('img')['title']  # Sport
            sex = cells[5].text.strip()  # Sex
            event = cells[6].text.strip()  # Event

            # Add the information to the list of athletes
            athletes.append({
                'medal': medal,
                'country': country,
                'athlete_name': athlete_name,
                'games': games,
                'sport': sport,
                'event': event,
                'sex' : sex
                })

        else: # Otherwise it's not an athlete, move to the next row
            next

    return pd.DataFrame(athletes)

url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"
# Try with one country - FRA (France)
html = GetHTML(url)

Getdataframe(html)

Response OK, continue.


,medal,country,athlete_name,games,sport,event,sex
0,1,France,Paul MASSON,Greece,Cycling Track,sprint individual,Men's
1,1,France,Paul MASSON,Greece,Cycling Track,1/3km time trial,Men's
2,1,France,Paul MASSON,Greece,Cycling Track,10km,Men's
3,1,France,Léon FLAMENG,Greece,Cycling Track,100km,Men's
4,1,France,Eugène-Henri GRAVELOTTE,Greece,Fencing,foil individual,Men's
...,...,...,...,...,...,...,...
884,2,France,SIMON Julia/FILLON MAILLET Quentin/CHEVALIER-B...,China,Biathlon,Relay mix,Mixed
885,2,France,LEDEUX Tess,China,Freestyle Skiing,Big air,Women's
886,2,France,TRESPEUCH Chloe,China,Snowboard,Snowboard Cross,Women's
887,3,France,FAIVRE Mathieu,China,Alpine Skiing,giant slalom,Men's


## <a id='toc1_4_'></a>[Data collect](#toc0_)

### <a id='toc1_4_1_'></a>[Extract one country data](#toc0_)

In [154]:
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=FRA&param_games=ALL&param_sport=ALL"

In [155]:
# Try with one country - FRA (France)
html = GetHTML(url)

Response OK, continue.


In [156]:
# Try to get the DF on one country - FRA (France)
df_medalists = Getdataframe(html)

df_medalists.sample(5)

,medal,country,athlete_name,games,sport,event,sex
827,1,France,Martin FOURCADE,Korea Republic,Biathlon,12.5km pursuit,Men's
766,3,France,Coralie BALMY/Mylene LAZARE/Charlotte BONNET/M...,Great Britain,Swimming,4x200m freestyle relay,Women's
634,2,France,Пуаре,United States,Biathlon,12.5km pursuit,Men's
15,1,France,Georges TAILLANDIER,France,Cycling Track,sprint individual,Men's
513,2,France,Franck BADIOU,Spain,Shooting,10m air rifle,Men's


### <a id='toc1_4_2_'></a>[Extract All countries data](#toc0_)

Scrape countries codes to loop on each of them (website url modification)  
Link of countries codes : <a href="http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"> Countries list</a>

In [157]:
# Countries list
url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=RefCountries&param_dbversion=&param_country=CIV&param_games=ALL&param_sport=ALL"

html = GetHTML(url)

Response OK, continue.


In [158]:
# Parsing text in Beautifulsoup object
soup = BeautifulSoup(html, "html5lib")

In [159]:
# Select the right table
table = soup.find_all("table", class_="simpletable", style="text-align:center;")[1] # 2nd table
rows = table.find_all("tr")
countries = []

for row in rows[2:]: # ignore the column names
    data = row.text.split()

    if len(data) == 5 and len(data[0]) == 3:
        countries.append(
            {
                'Code':data[0], 
                'Country':data[1], 
                'Continent':data[2], 
                'Firstparticipation':data[3], 
                'Lastparticipation':data[4]
            }
        )
    else :
        next

df_countries = pd.DataFrame(countries)

In [160]:
df_countries.sample(5)

,Code,Country,Continent,Firstparticipation,Lastparticipation
84,LBN,Lebanon,Asia,1948,2022
4,ANG,Angola,Africa,1980,2021
7,ARU,Aruba,America,1988,2021
11,BAH,Bahamas,America,1952,2021
38,DEN,Denmark,Europe,1896,2022


### <a id='toc1_4_3_'></a>[Extract all data](#toc0_)

In [161]:
i = 0
if i == 1:
    # Run scraping on all countries 
    code_countries = df_countries["Code"]
    name_countries = df_countries["Country"]
    i = 0

    for country, country_name in zip(code_countries, name_countries) :
        url = "http://olympanalyt.com/OlympAnalytics.php?param_pagetype=Medals&param_dbversion=&param_country=" + country + "&param_games=ALL&param_sport=ALL"
        html = GetHTML(url)
        df_temp = Getdataframe(html)
        if i == 0 :
            df_olympic = df_temp
            i += 1
        else : 
            df_olympic = pd.concat([df_olympic, df_temp], ignore_index=True)

        
        print("for : {}".format(country_name))

In [162]:
#df_olympic.to_csv("DB_olympic.csv", sep=";", index=False)

In [163]:
df_olympic = pd.read_csv("DB_olympic.csv", delimiter=";")
df_olympic.fillna("", inplace=True)

In [164]:
df_olympic.describe(include="object")

,country,athlete_name,games,sport,event,sex
count,12317,12317,12317,12317,12317,12317
unique,119,9998,25,76,659,3
top,Germany,,United States,Athletics,individual,Men's
freq,922,309,1528,1464,357,8052


### <a id='toc1_4_4_'></a>[Description of data](#toc0_)

In [165]:
# Looking for none-latin names
latin_regex = r'[A-Za-zÀ-ÿ]'

df_filtered = df_olympic[df_olympic["athlete_name"].str.contains(latin_regex, regex=True)]
df_none_latine = df_olympic[~df_olympic["athlete_name"].str.contains(latin_regex, regex=True)]

In [166]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10093 entries, 0 to 12316
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   medal         10093 non-null  int64 
 1   country       10093 non-null  object
 2   athlete_name  10093 non-null  object
 3   games         10093 non-null  object
 4   sport         10093 non-null  object
 5   event         10093 non-null  object
 6   sex           10093 non-null  object
dtypes: int64(1), object(6)
memory usage: 630.8+ KB


In [167]:
df_filtered.sample(5)

,medal,country,athlete_name,games,sport,event,sex
2800,2,China,Shuang GUO,Great Britain,Cycling Track,Keirin,Women's
5452,3,Germany,Christian TRÖGER/Mark PINGER/Bengt ZIKARSKY/Di...,Spain,Swimming,4x100m freestyle relay,Men's
2370,3,China,Mei-Su LI,Korea Republic,Athletics,shot put,Women's
2283,1,Canada,FILLIER Sarah/JOHNSTON Rebecca/MALTAIS Emma/NU...,China,Ice Hockey,ice hockey,Women's
741,1,Austria,Trude JOCHUM-BEISER,Switzerland,Alpine Skiing,Alpine combined,Women's


In [168]:
df_none_latine.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2224 entries, 6 to 12278
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   medal         2224 non-null   int64 
 1   country       2224 non-null   object
 2   athlete_name  2224 non-null   object
 3   games         2224 non-null   object
 4   sport         2224 non-null   object
 5   event         2224 non-null   object
 6   sex           2224 non-null   object
dtypes: int64(1), object(6)
memory usage: 139.0+ KB


In [169]:
df_none_latine.sample(5)

,medal,country,athlete_name,games,sport,event,sex
10554,2,Slovakia,Княжовицки,United States,Canoe Sprint,C-1 500m (canoe single),Men's
2536,2,China,Лин Цзе,Australia,Gymnastics Artistic,uneven bars,Women's
4755,2,France,Клинье,United States,Cycling Track,individual pursuit,Women's
8081,1,Japan,Кейджи Сузуки,Greece,Judo,+ 100kg (heavyweight),Men's
7438,3,Italy,Давиде Руммоло,Australia,Swimming,200m breaststroke,Men's


## <a id='toc1_5_'></a>[Transform](#toc0_)

### athlete_name

Translate non-latin names in latin

In [170]:
# Translate none-latin in Latin names
from unidecode import unidecode

df_olympic["athlete_name"] = df_olympic["athlete_name"].apply(lambda s: unidecode(s))

In [171]:
df_olympic.sample(5)

,medal,country,athlete_name,games,sport,event,sex
3808,3,Finland,Lennart HANNELIUS,France,Shooting,25m rapid fire pistol,Men's
4110,1,Finland,Ari-Pekka NIKKOLA/Risto LAAKKONEN/Mika LAITINE...,France,Ski Jumping,Large Hill team,Men's
2779,1,China,Jike ZHANG,Great Britain,Table Tennis,singles,Men's
9441,1,Norway,Tormod KNUTSEN,Austria,Nordic Combined,individual,Men's
10673,3,Spain,Sergio LOPEZ MIRO,Korea Republic,Swimming,200m breaststroke,Men's


In [172]:
# Check if there are more than 1 athlete
def is_team(strings):
    """
    Information : Athletes names are separate by "/" if they are more than one
    Return: Boolean
    """
    return len(strings.split("/")) > 1

# Create a new colonne for team information
df_olympic["team"] = ["Yes" if is_team(athlete) else "No" for athlete in df_olympic["athlete_name"]]

# Team athletes
#df_final_team = df_olympic[df_olympic["athlete_name"].apply(is_team)]

# Solo athlete
#df_final_solo = df_olympic[~df_olympic["athlete_name"].apply(is_team)]

In [173]:
df_olympic

,medal,country,athlete_name,games,sport,event,sex,team
0,3,Afghanistan,Rohullah NIKPAI,China,Taekwondo,- 58 kg,Men's,No
1,3,Afghanistan,Rohullah NIKPAI,Great Britain,Taekwondo,58 - 68 kg,Men's,No
2,3,Algeria,Mohamed ZAOUI,United States,Boxing,71 - 75kg (middleweight),Men's,No
3,3,Algeria,Mustapha MOUSSA,United States,Boxing,75 - 81kg (light-heavyweight),Men's,No
4,1,Algeria,Hassiba BOULMERKA,Spain,Athletics,1500m,Women's,No
...,...,...,...,...,...,...,...,...
12312,3,Mixed Team,Robert Fournier-Sarloveze (FRA)/Frederick Agne...,France,Polo,polo,Men's,Yes
12313,1,Mixed Team,Ramon FONST (CUB)/Albertson VAN ZO POST (USA)/...,United States,Fencing,foil team,Men's,Yes
12314,1,Mixed Team,Philipp KASSEL (USA)/Max HESS (USA)/John GRIEB...,United States,Gymnastics Artistic,team competition,Men's,Yes
12315,2,Mixed Team,James LIGHTBODY (USA)/Lacey HEARN (USA)/Albert...,United States,Athletics,4miles team,Men's,Yes


### Empty values

Looking for **empty** values (there are no missing values).

In [174]:
df_olympic.select_dtypes("object").columns

Index(['country', 'athlete_name', 'games', 'sport', 'event', 'sex', 'team'], dtype='object')

In [175]:
for col in df_olympic.select_dtypes("object").columns:
    print(col)

country
athlete_name
games
sport
event
sex
team


In [179]:
len(df_olympic[df_olympic["country"] == ""]["country"])

0

In [180]:
# Looking for empty values on all columns
for col in df_olympic.select_dtypes("object").columns:

    col_temp = df_olympic[df_olympic[col] == ""][col]

    if len(col_temp) != 0:
        print(col)

athlete_name


In [181]:
df_olympic[df_olympic["athlete_name"] == ""].sample(5)

,medal,country,athlete_name,games,sport,event,sex,team
7414,1,Italy,,Australia,Rowing,quadruple sculls without coxswain,Men's,No
503,3,Australia,,Greece,Rowing,quadruple sculls without coxswain,Women's,No
7463,2,Italy,,Greece,Basketball,basketball,Men's,No
4758,2,France,,United States,Rowing,four without coxswain (4-),Men's,No
5654,3,Germany,,Australia,Rowing,quadruple sculls without coxswain,Men's,No


All missing athlete name are teams.  
Fill these by country name.

In [182]:
# Change values if the name is empty
df_olympic.loc[df_olympic["athlete_name"] == "", "team"] = "Yes"

# Fill empty values by the country
df_olympic["athlete_name"] = [name if name != "" else country for name, country in zip(df_olympic["athlete_name"], df_olympic["country"])]

In [183]:
#Check
df_olympic[df_olympic["athlete_name"] == ""]

,medal,country,athlete_name,games,sport,event,sex,team


In [188]:
df_olympic.sport.unique()

array(['Taekwondo', 'Boxing', 'Athletics', 'Judo', 'Polo', 'Swimming',
       'Football', 'Fencing', 'Rowing', 'Sailing', 'Shooting',
       'Weightlifting', 'Equestrian Eventing', 'Tennis', 'Volleyball',
       'Hockey', 'Basketball', 'Cycling Track', 'Rugby',
       'Wrestling Greco-Roman', 'Wrestling Freestyle',
       'Gymnastics Artistic', 'Diving', 'Canoe Sprint', 'Cycling Road',
       'Canoe Slalom', 'Short Track Speed Skating', 'Beach Volleyball',
       'Softball^(1996-2008)', 'Alpine Skiing', 'Archery', 'Water Polo',
       'Trampoline', 'Triathlon', 'Freestyle Skiing',
       'Baseball^(1992-2008)', 'Snowboard', 'Cycling BMX Racing',
       'Modern Pentathlon', 'Cycling BMX Freestyle', 'Skateboarding',
       'Marathon swimming', 'Surfing', 'Skeleton', 'Figure skating',
       'Handball', 'Equestrian Dressage', 'Speed skating', 'Ski Jumping',
       'Luge', 'Bobsleigh', 'Nordic Combined', 'Equestrian Jumping',
       'Cross Country Skiing', 'Biathlon', 'Karate', 'Sport Clim

### Event

In [187]:
for event in df_olympic.event.unique():
    if "kg" in event:
        print(event)

- 58 kg
58 - 68 kg
71 - 75kg (middleweight)
75 - 81kg (light-heavyweight)
54 - 57kg (featherweight)
57 - 60kg (lightweight)
60 - 63.5kg (light-welterweight)
81 - 90kg (middleweight)
48 - 52kg (half-lightweight)
57.15 - 61.24kg (lightweight)
61.24 - 66.68kg (welterweight)
53.52 - 57.15kg (featherweight)
66.68 - 72.57kg (middleweight)
72.57 - 79.38kg (light-heavyweight)
+ 79.38kg (heavyweight)
- 51kg (flyweight)
+ 80kg (heavyweight)
73 - 80kg (light-heavyweight)
67 - 71kg (light-middleweight)
+ 90kg (heavyweight)
63.5 - 67kg (welterweight)
- 48kg (extra-lightweight)
68 - 80 kg
48 - 52kg (flyweight)
- 48kg (light-flyweight)
69 - 77kg (middleweight)
77 - 85kg (light-heavyweight)
- 55kg
96 - 120kg
66 - 74kg
84 - 96kg
98kg
94 - 105kg (heavyweight)
+ 105kg (super heavyweight)
66kg
109kg
97kg
57 - 63kg (lightweight)
79 - 87kg (light-heavyweight)
67 - 73kg (welterweight)
+ 87kg (heavyweight)
60 - 67.5kg (lightweight)
51 - 54kg (bantamweight)
+ 110kg (super heavyweight)
75 - 82.5kg (light-heavyw

To do :
- Get simple sport list
- Get simple event list (weight, lenght, heigh, etc. split ?)
- ~~Look for empty cells on each column (sometimes filled with country name)~~

1. git branch **nom_de_branche**
2. git checkout **nom_de_branche**
3. *changement*
4. git add . 
5. git commit -m "*commentaire du commit*"

6. git push --set-upstream origin notice_alex
6. **bis** git push (*pour pousser sur le branche*)
